# Making our first maps

We will start simple.  

And introduce ourselves to some new Python libraries: `folium`, `geopandas`, and `contextily`!

* [folium documentation](https://python-visualization.github.io/folium/quickstart.html)
* [geopandas documentation](https://geopandas.readthedocs.io/en/latest/gallery/index.html)
* [contextily documentation](https://contextily.readthedocs.io/en/latest/intro_guide.html)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Our new mapping libraries
import folium
import geopandas as gpd
import contextily as ctx

I have made an ultra-simple csv data file that contains the locations of UCLA and 3 airports, along with a fictitious numerical value called "Score".

In [ ]:
# note that you will need to specify the proper data directory 
# this path assumes your current directory is UCLAX-24Fall-EDA/Notebooks/Week06/ 

df = pd.read_csv('../../Data/airport.csv')

In [ ]:
df

Making a map with `folium` is easy.

In [ ]:
# default folium map
m = folium.Map()
m

To center it on our region of interest, let's first get the average latitude and longitude values.

In [ ]:
# average latitude
latitude = df['Latitude'].mean()
latitude

In [ ]:
# average longitude
longitude = df['Longitude'].mean()
longitude

In [ ]:
# Here is a map centered on the average lat/lon calculated above
m = folium.Map(location=[latitude,longitude])
m

To retrieve the points of interest, we can iterate over the rows of our dataframe.

In [ ]:
for index, row in df.iterrows():
   print([row.Latitude, row.Longitude])

In [ ]:
# now we'll add the points to the map
for index, row in df.iterrows():
   folium.Marker([row.Latitude, row.Longitude]).add_to(m)
m

We are also going to be working with geometrical objects with the Python library called `geopandas`.  More on this particular library soon!

Whenever you bring data into a Jupyter notebook that is in a json or csv format, if the data contains separate columns for latitude and longitude, you can convert the data into a "geo"dataframe for mapping purposes.

*Note*: If the data has latitude and longitude coordinates, it is a geographic coordinate system (GCS) with a crs code epsg:4326.

In [ ]:
from shapely.geometry import Point

In [ ]:
geometry = [Point(loc) for loc in zip(df.Longitude, df.Latitude)]
gdf = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry=geometry)

In [ ]:
gdf

In [ ]:
gdf.crs

In [ ]:
gdf.plot()

In [ ]:
# reproject to Web Mercator to layer our points on top of a base map
gdf_web_mercator = gdf.to_crs(epsg=3857)

In [ ]:
gdf_web_mercator.plot()

In [ ]:
fig,ax = plt.subplots(figsize=(17, 17))
gdf_web_mercator.plot(ax=ax,column='Score',
                      cmap='rainbow',
                      edgecolor='black',
                      legend=True)
ctx.add_basemap(ax, 
                source=ctx.providers.OpenStreetMap.Mapnik)

We will later make a choropleth map with Tableau, so let's make one here to see it done too.

*Note*: "choro"pleth, not "chloro"pleth.

In [ ]:
url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
state_geo = f"{url}/us-states.json"
state_unemployment = f"{url}/US_Unemployment_Oct2012.csv"
state_data = pd.read_csv(state_unemployment)

In [ ]:
state_geo

In [ ]:
state_data

In [ ]:
m = folium.Map(location=[48, -102], zoom_start=3)
m

In [ ]:
folium.Choropleth(
    geo_data=state_geo,
    data=state_data,
    columns=["State", "Unemployment"],
    legend_name="Unemployment Rate (%)",
    key_on="feature.id",
    fill_color="YlGn",
).add_to(m)

m